In [ ]:
from fastai.vision.all import *
import torchvision.models as models
import matplotlib.pyplot as plt

from utils import label_func

print("cuda.is_available: {}".format(torch.cuda.is_available()))
set_seed(1234)

In [ ]:
def load_data(path, size=224):
    fnames = get_image_files(path)
    item_tfms = [ToTensor()]
    if size is not None:
        item_tfms.append(Resize(size))
    dls = ImageDataLoaders.from_path_func(
        path,
        fnames,
        label_func,
        item_tfms=item_tfms,
        bs=64,
    )
    return dls
    
path = "dogs/train/"

dls = load_data(path)

In [ ]:
dls.show_batch()

## Train classifier for dog breeds

In [ ]:
learn = cnn_learner(dls, models.resnext50_32x4d, cut=-1, pretrained=True, metrics=accuracy)

In [ ]:
lr_min = learn.lr_find()
print(lr_min.valley)

In [ ]:
learn.fit_one_cycle(
    5,
    lr_min.valley,
    cbs=[SaveModelCallback(monitor="accuracy")]
)

In [ ]:
learn.recorder.plot_loss(skip_start=200)

In [ ]:
learn.recorder.plot_sched()

In [ ]:
learn.show_results()

In [ ]:
learn.export("exported_resnext50_32x4d.pickle")

## Use trained model to predict the class of some images

In [ ]:
paths = get_image_files("dogs/train/")

In [ ]:
def test(path):
    img = PILImage.create(path)
    pred, pred_ix, probs = learn.predict(img)
    print(f"Groundtruth class: {path.parent.name}")
    print(f"Predicted class:   {pred} with prob {probs[pred_ix]}")
    show_image(img)

In [ ]:
test(paths[0])

In [ ]:
test(paths[200])

In [ ]:
test(paths[400])